<a href="https://colab.research.google.com/github/hardik-xi11/military-ml-model/blob/main/military_base_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install official Ultralytics and Roboflow libraries
!pip install -q ultralytics roboflow supervision
import ultralytics
ultralytics.checks() # Ensure GPU is active

Ultralytics 8.3.63 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 22.6/107.7 GB disk)


In [2]:
from roboflow import Roboflow

# Connect to your Roboflow project
rf = Roboflow(api_key="Sh4JCfC1au7aMELChD2M")
project = rf.workspace("soviet-ehzes").project("military-base-object-detection-ewcmm")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
from ultralytics import YOLO

# Load pre-trained YOLO11 Small (Professional 'sweet spot' for defense)
model = YOLO('yolo11s.pt')

# Launch Optimized Training
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=100,            # 100 epochs for production-level accuracy
    imgsz=800,             # Reduced from 1024 to 800 to save RAM
    batch=8,               # Explicitly set to 8 to avoid RAM exhaustion
    workers=2,             # Lower workers = Lower RAM usage
    name='indian_defense_sentinel',
    augment=True,          # Helps detection of 'Camouflage' targets
    patience=30,           # Stops training if model stops improving
    cache=False            # DO NOT set to True; it will crash RAM for large datasets
)

100%|██████████| 18.4M/18.4M [00:00<00:00, 145MB/s]


New https://pypi.org/project/ultralytics/8.3.241 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/Military-Base-Object-Detection-1/data.yaml, epochs=100, time=None, patience=30, batch=8, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=indian_defense_sentinel2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False

train: Scanning /content/Military-Base-Object-Detection-1/train/labels... 8415 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8415/8415 [00:13<00:00, 621.09it/s]


train: New cache created: /content/Military-Base-Object-Detection-1/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


Argument(s) 'quality_lower' are not valid for transform ImageCompression
'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
val: Scanning /content/Military-Base-Object-Detection-1/valid/labels... 1911 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1911/1911 [00:02<00:00, 715.27it/s]

val: New cache created: /content/Military-Base-Object-Detection-1/valid/labels.cache


Plotting labels to /content/yolov12/runs/detect/indian_defense_sentinel2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 800 train, 800 val
Using 0 dataloader workers
Logging results to /content/yolov12/runs/detect/indian_defense_sentinel2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      1.623      4.289       1.62         79        800:  32%|███▏      | 332/1052 [2:24:49<5:30:32, 27.55s/it]

In [ ]:
import os
from IPython.display import Image, display

# Directory where the latest training run results are stored
results_dir = '/content/runs/detect/indian_defense_sentinel/'

print("--- Professional Metrics Summary ---")
# Check if confusion matrix exists before plotting
if os.path.exists(f'{results_dir}confusion_matrix.png'):
    display(Image(filename=f'{results_dir}confusion_matrix.png', width=800))

# Plot results.png (shows training vs validation loss curves)
if os.path.exists(f'{results_dir}results.png'):
    display(Image(filename=f'{results_dir}results.png', width=800))

In [ ]:
import glob
import random

# Load the best weights from your training session
best_model = YOLO(f'{results_dir}weights/best.pt')

# Select random images from the 'test' set
test_images = glob.glob(f"{dataset.location}/test/images/*.jpg")
sample = random.choice(test_images)

# Run tactical prediction
results = best_model.predict(source=sample, save=True, conf=0.35)

# TACTICAL ALERT LOGIC (Intent Engine)
for result in results:
    detected = [best_model.names[int(box.cls)] for box in result.boxes]
    print(f"Sensors Detected: {detected}")

    if "Drone" in detected and "Soldier" in detected:
        print("🚨 TACTICAL ALERT: Possible UAV Surveillance detected on Personnel!")
    if "Camouflage" in detected and "Rifle" not in detected:
        print("⚠️ INFILTRATION WARNING: Stealth asset detected in vicinity.")
    if "Fire" in detected or "Smoke" in detected:
        print("🔥 INFRASTRUCTURE ALERT: Battlefield event detected.")

In [ ]:
# Export for real-world deployment on drones or tablets
best_model.export(format="onnx")
print("Export complete. Download your model from the runs/detect/.../weights folder.")